In [1]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

Connect SQL

In [4]:
db, conn = pgconnect(credentials)

Connected successfully.


In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
import numpy as np

Open datasets

In [6]:
regions = gpd.read_file('SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp')
businesses = pd.read_csv('Businesses.csv')
stops = pd.read_csv('Stops.txt')
polls = pd.read_csv('PollingPlaces2019.csv')
school_future = gpd.read_file('catchments/catchments_future.shp')
school_primary = gpd.read_file('catchments/catchments_primary.shp')
school_secondary = gpd.read_file('catchments/catchments_secondary.shp')
population = pd.read_csv('Population.csv')
income = pd.read_csv('Income.csv')

Ensure every row in geometry is represented as multipolygons

In [7]:
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

srid = 4326
regions.dropna(inplace=True)
regions['geom'] = regions['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
regions = regions.drop(columns="geometry")  # deleting the old copy

school_future['geom'] = school_future['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))
school_future = school_future.drop(columns="geometry")

school_primary['geom'] = school_primary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))
school_primary = school_primary.drop(columns="geometry")

school_secondary['geom'] = school_secondary['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))
school_secondary = school_secondary.drop(columns="geometry")

Create a column 'geom' for stops

In [8]:
stops['geom'] = gpd.points_from_xy(stops.stop_lon, stops.stop_lat)
stops = stops.drop(columns=['stop_lat', 'stop_lon'])
stops['geom'] = stops['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

Rename columns because some of the column names start with number or is uppercase

In [9]:
def col_converter_regions(col_names):
    col_dict = {}
    for c in col_names:
        new_c = c.lower() #convert every column name to lowercase
        if new_c[-2:] == '21': # if column names end with 21, remove it
            new_c = new_c[:-2]
        if new_c == 'loci_uri':
            new_c = 'url'
        col_dict[c] = new_c
    return col_dict

def col_converter_school(col_names):
    col_dict = {}
    for c in col_names:
        col_dict[c] = c.lower()
    return col_dict

col_names = regions.columns.values.tolist()
regions = regions.rename(columns=col_converter_regions(col_names))

col_names = school_future.columns.tolist()
school_future = school_future.rename(columns=col_converter_school(col_names))

col_names = school_primary.columns.tolist()
school_primary = school_primary.rename(columns=col_converter_school(col_names))

col_names = school_secondary.columns.tolist()
school_secondary = school_secondary.rename(columns=col_converter_school(col_names))

businesses = businesses.rename(columns={'0_to_50k_businesses':'b_0_to_50k', '50k_to_200k_businesses':'b_50k_to_200k', '200k_to_2m_businesses':'b_200k_to_2m'})
businesses = businesses.rename(columns={'2m_to_5m_businesses':'b_2m_to_5m', '5m_to_10m_businesses':'b_5m_to_10m', '10m_or_more_businesses':'b_10m_or_more'})

population = population.rename(columns={'0-4_people':'p_0_to_4', '5-9_people':'p_5_to_9', '10-14_people':'p_10_to_14', '15-19_people':'p_15_to_19'})
population = population.rename(columns={'20-24_people':'p_20_to_24', '25-29_people':'p_25_to_29', '30-34_people':'p_30_to_34', '35-39_people':'p_35_to_39'})
population = population.rename(columns={'40-44_people':'p_40_to_44', '45-49_people':'p_45_to_49', '50-54_people':'p_50_to_54', '55-59_people':'p_55_to_59'})
population = population.rename(columns={'60-64_people':'p_60_to_64', '65-69_people':'p_65_to_69', '70-74_people':'p_70_to_74', '75-79_people':'p_75_to_79'})
population = population.rename(columns={'80-84_people':'p_80_to_84', '85-and-over_people':'p_85_and_over'})

polls = polls.rename(columns={'FID':'fid'})

Replace some NA values

In [10]:
income = income.replace('np', None)

Create Schema

In [11]:
conn.execute("""
DROP TABLE IF EXISTS regions;
CREATE TABLE regions (
    sa2_code VARCHAR(9),
    sa2_name VARCHAR(100),
    chg_flag CHAR,
    chg_lbl VARCHAR(20),
    sa3_code VARCHAR(5),
    sa3_name VARCHAR(100),
    sa4_code VARCHAR(3),
    sa4_name VARCHAR(100),
    gcc_code VARCHAR(5),
    gcc_name VARCHAR(100),
    ste_code CHAR,
    ste_name VARCHAR(100),
    aus_code VARCHAR(3),
    aus_name VARCHAR(100),
    areasqkm FLOAT,
    url VARCHAR(100),
    geom GEOMETRY(MULTIPOLYGON,4326)
);
""")

conn.execute("""
DROP TABLE IF EXISTS businesses;
CREATE TABLE businesses (
    industry_code CHAR, 
    industry_name VARCHAR(100),
    sa2_code VARCHAR(9),
    sa2_name VARCHAR(100),
    b_0_to_50k INTEGER,
    b_50k_to_200k INTEGER,
    b_200k_to_2m INTEGER,
    b_2m_to_5m INTEGER,
    b_5m_to_10m INTEGER,
    b_10m_or_more INTEGER,
    total_businesses INTEGER
);
""")
#b_0_to_50k is the number of businesses' income < 50k

conn.execute("""
DROP TABLE IF EXISTS stops;
CREATE TABLE stops (
    stop_id VARCHAR(10),
    stop_code VARCHAR(10),
    stop_name VARCHAR(100),
    location_type FLOAT,
    parent_station VARCHAR(10),
    wheelchair_boarding CHAR,
    platform_code VARCHAR(10),
    geom GEOMETRY(POINT, 4326)
);
""")

conn.execute("""
DROP TABLE IF EXISTS polls;
CREATE TABLE polls (
    fid VARCHAR(100),
    state VARCHAR(3),
    division_id INTEGER,
    division_name VARCHAR(100),
    polling_place_id INTEGER,
    polling_place_type_id INTEGER,
    polling_place_name VARCHAR(100),
    premises_name VARCHAR(100),
    premises_address_1 VARCHAR(100),
    premises_address_2 VARCHAR(100),
    premises_address_3 VARCHAR(100),
    premises_suburb VARCHAR(100),
    premises_state_abbreviation VARCHAR(3),
    premises_post_code INTEGER,
    latitude FLOAT,
    longitude FLOAT,
    the_geom GEOMETRY(POINT,4326)
);
""")

conn.execute("""
DROP TABLE IF EXISTS school_future;
CREATE TABLE school_future (
    use_id VARCHAR(4),
    catch_type VARCHAR(20),
    use_desc VARCHAR(100),
    add_date INTEGER,
    kindergart INTEGER,
    year1 INTEGER,
    year2 INTEGER,
    year3 INTEGER,
    year4 INTEGER,
    year5 INTEGER,
    year6 INTEGER,
    year7 INTEGER,
    year8 INTEGER,
    year9 INTEGER,
    year10 INTEGER,
    year11 INTEGER,
    year12 INTEGER,
    geom GEOMETRY(MULTIPOLYGON,4326)
);
""")

conn.execute("""
DROP TABLE IF EXISTS school_primary;
CREATE TABLE school_primary (
    use_id VARCHAR(4),
    catch_type VARCHAR(20),
    use_desc VARCHAR(100),
    add_date INTEGER,
    kindergart CHAR,
    year1 CHAR,
    year2 CHAR,
    year3 CHAR,
    year4 CHAR,
    year5 CHAR,
    year6 CHAR,
    year7 CHAR,
    year8 CHAR,
    year9 CHAR,
    year10 CHAR,
    year11 CHAR,
    year12 CHAR,
    priority CHAR,
    geom GEOMETRY(MULTIPOLYGON,4326)
);
""")

conn.execute("""
DROP TABLE IF EXISTS school_secondary;
CREATE TABLE school_secondary (
    use_id VARCHAR(4),
    catch_type VARCHAR(20),
    use_desc VARCHAR(100),
    add_date INTEGER,
    kindergart CHAR,
    year1 CHAR,
    year2 CHAR,
    year3 CHAR,
    year4 CHAR,
    year5 CHAR,
    year6 CHAR,
    year7 CHAR,
    year8 CHAR,
    year9 CHAR,
    year10 CHAR,
    year11 CHAR,
    year12 CHAR,
    priority CHAR,
    geom GEOMETRY(MULTIPOLYGON,4326)
);
""")

conn.execute("""
DROP TABLE IF EXISTS population;
CREATE TABLE population (
    sa2_code VARCHAR(9),
    sa2_name VARCHAR(100),
    p_0_to_4 INTEGER,
    p_5_to_9 INTEGER,
    p_10_to_14 INTEGER,
    p_15_to_19 INTEGER,
    p_20_to_24 INTEGER,
    p_25_to_29 INTEGER,
    p_30_to_34 INTEGER,
    p_35_to_39 INTEGER,
    p_40_to_44 INTEGER,
    p_45_to_49 INTEGER,
    p_50_to_54 INTEGER,
    p_55_to_59 INTEGER,
    p_60_to_64 INTEGER,
    p_65_to_69 INTEGER,
    p_70_to_74 INTEGER,
    p_75_to_79 INTEGER,
    p_80_to_84 INTEGER,
    p_85_and_over INTEGER,
    total_people INTEGER
);
"""
)
#p_0_to_4 is the number of people age from 0 to 4

conn.execute("""
DROP TABLE IF EXISTS income;
CREATE TABLE income (
    sa2_code VARCHAR(9),
    sa2_name VARCHAR(100),
    earners INTEGER,
    median_age INTEGER,
    median_income INTEGER,
    mean_income INTEGER
);
"""
)

Importing dataframes to sql

In [12]:
regions.to_sql('regions', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
businesses.to_sql('businesses', conn, if_exists='append', index=False)
stops.to_sql('stops', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})
polls.to_sql('polls', conn, if_exists='append', index=False)
school_future.to_sql('school_future', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
school_primary.to_sql('school_primary', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
school_secondary.to_sql('school_secondary', conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
population.to_sql('population', conn, if_exists='append', index=False)
income.to_sql('income', conn, if_exists='append', index=False)

In [13]:
query(conn, "select * from school_future")

,use_id,catch_type,use_desc,add_date,kindergart,year1,year2,year3,year4,year5,year6,year7,year8,year9,year10,year11,year12,geom
0,8416,HIGH_COED,Ku-ring-gai HS,20230114,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,0106000020E61000000100000001030000000100000090...
1,8161,HIGH_BOYS,Randwick BHS,20200220,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,0106000020E6100000010000000103000000010000006F...
2,8539,HIGH_COED,SSC Blackwattle Bay,20220609,0,0,0,0,0,0,0,0,0,0,0,2024,2024,0106000020E610000001000000010300000001000000E3...
3,8286,HIGH_COED,Mt Annan HS,20220301,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,0106000020E61000000100000001030000000100000099...
4,8400,HIGH_COED,St Ives HS,20230114,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,0106000020E61000000100000001030000000100000060...
5,8555,HIGH_COED,Rose Bay SC,20200220,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,0106000020E6100000010000000103000000010000000C...
6,8556,CENTRAL_HIGH,Alexandria Park CS,20200220,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,0106000020E61000000100000001030000000100000028...
7,8584,HIGH_COED,Elizabeth Macarthur HS,20220301,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,0106000020E610000001000000010300000001000000BB...
8,8913,HIGH_COED,Inner Sydney HS,20200220,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2025,0106000020E61000000100000001030000000100000076...
9,8135,HIGH_COED,Parramatta HS,20230112,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,0106000020E6100000010000000103000000010000001F...
